## 冰湖遊戲 (Frozen lake) 策略評估計算 (參考 15-10 Policy_Evaluation)

In [142]:
import gymnasium as gym
import numpy as np

In [143]:
# MAPS = {
#     "4x4": ["SFFF", "FHFH", "FFFH", "HFFG"],
# }

In [144]:
# 創建FrozenLake環境
env = gym.make("FrozenLake-v1")
# 獲取狀態和行動的維度 (現在 FrozenLake 不支援 nS, nA ,以及沒有 env.shape)
n_states = env.observation_space.n
n_actions = env.action_space.n
state_shape = env.observation_space.n

In [145]:
print('{狀態: {行動: [ (轉移機率, 下一個狀態, 獎勵, 是否到達終點), (轉移機率, 下一個狀態, 獎勵, 是否到達終點)]}}') #字典裡面包字典,又包 list
env.P  #Bellman 公式,對照 P 的部分 (講義 p.91 : file:///D:/Python/Python3/Class%2017/0_RL/RL_Introduction.pdf)
# 行動: 0: Move left, 1: Move down, 2: Move right, 3: Move up
# 由於一開始沒有定義 slipper 等條件, 所以會滑行 (slipper 預設為 true),因此,測試結果才會有兩次 0,然後跑到 4 (或許是滑行)

{狀態: {行動: [ (轉移機率, 下一個狀態, 獎勵, 是否到達終點), (轉移機率, 下一個狀態, 獎勵, 是否到達終點)]}}


{0: {0: [(0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 4, 0.0, False)],
  1: [(0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 4, 0.0, False),
   (0.3333333333333333, 1, 0.0, False)],
  2: [(0.3333333333333333, 4, 0.0, False),
   (0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False)],
  3: [(0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 0, 0.0, False)]},
 1: {0: [(0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 5, 0.0, True)],
  1: [(0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 5, 0.0, True),
   (0.3333333333333333, 2, 0.0, False)],
  2: [(0.3333333333333333, 5, 0.0, True),
   (0.3333333333333333, 2, 0.0, False),
   (0.3333333333333333, 1, 0.0, False)],
  3: [(0.3333333333333333, 2, 0.0, False),
   (0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False)]},
 2:

## 策略評估函數

In [146]:
def policy_eval(policy, env, epoch=1, discount_factor=1.0):
    # 狀態值函數初始化
    V = np.zeros(n_states)
    V1 = np.copy(V)
    no = 0
    while no < epoch:
        # 更新每個狀態值的函數
        for s in range(n_states):
            v = 0
            # 計算每個行動後的狀態值函數
            for a, action_prob in enumerate(policy[s]):
                # 取得所有可能的下一狀態值
                for  prob, next_state, reward, done in env.P[s][a]:
                    # 狀態值函數公式，依照所有可能的下一狀態值函數加總 
                    v += action_prob * prob * (reward + 
                                   discount_factor * V[next_state])
            V1[s] = v
        V = np.copy(V1)
        no+=1
    return np.array(V)

## 訓練 1 週期

In [150]:
# 隨機策略，機率均等
random_policy = np.ones([n_states, n_actions]) / n_actions
# 評估
v = policy_eval(random_policy, env, 1)
print("4x4 狀態值函數:")
print(v.reshape(state_shape))

4x4 狀態值函數:
[0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
 0.25 0.  ]


## 訓練 2 週期

In [151]:
v = policy_eval(random_policy, env, 2)
print("4x4 狀態值函數:")
print(v.reshape(state_shape))

4x4 狀態值函數:
[0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
 0.0625 0.     0.     0.0625 0.3125 0.    ]


## 訓練 3 週期

In [152]:
v = policy_eval(random_policy, env, 3)
print("4x4 狀態值函數:")
print(v.reshape(state_shape))

4x4 狀態值函數:
[0.       0.       0.       0.       0.       0.       0.015625 0.
 0.       0.03125  0.078125 0.       0.       0.09375  0.359375 0.      ]


## 完整的策略評估函數

In [153]:
# 策略評估函數
def policy_eval(policy, env, discount_factor=1.0, theta=0.00001):
    # 狀態值函數初始化
    V = np.zeros(n_states)
    V1 = np.copy(V)
    while True:
        delta = 0
        # 更新每個狀態值的函數
        for s in range(n_states):
            v = 0
            # 計算每個行動後的狀態值函數
            for a, action_prob in enumerate(policy[s]):
                # 取得所有可能的下一狀態值
                for  prob, next_state, reward, done in env.P[s][a]:
                    # 狀態值函數公式，依照所有可能的下一狀態值函數加總 
                    v += action_prob * prob * (reward + 
                                   discount_factor * V[next_state])
            # 比較更新前後的差值，取最大值
            delta = max(delta, np.abs(v - V[s]))
            V1[s] = v
        V = np.copy(V1)
        # 若最大差值 < 門檻值，則停止評估
        if delta < theta:
            break
    return np.array(V)

In [154]:
# 隨機策略，機率均等
random_policy = np.ones([n_states, n_actions]) / n_actions
# 評估
v = policy_eval(random_policy, env)

print("4x4 狀態值函數:")
print(v.reshape(state_shape))

4x4 狀態值函數:
[0.01390102 0.01160346 0.02092871 0.01045776 0.0162259  0.
 0.04074201 0.         0.03479274 0.08816183 0.14204605 0.
 0.         0.1758143  0.43928541 0.        ]


## 策略改善函數 (參考 15-11 Policy_Iteration_FrozenLake)

In [155]:
def policy_improvement(env, policy_eval_fn=policy_eval, discount_factor=1.0):
    # 計算行動值函數
    def one_step_lookahead(state, V):
        A = np.zeros(n_actions)
        for a in range(n_actions):
            for prob, next_state, reward, done in env.P[state][a]:
                A[a] += prob * (reward + discount_factor * V[next_state])
        return A
    
    # 一開始採隨機策略，往上/下/左/右走的機率(π)均等
    policy = np.ones([n_states, n_actions]) / n_actions
    
    while True:
        # 策略評估
        V = policy_eval_fn(policy, env, discount_factor)
        
        # 若要改變策略，會設定 policy_stable = False
        policy_stable = True
        
        for s in range(n_states):
            # 依 P 選擇最佳行動
            chosen_a = np.argmax(policy[s])
            
            # 計算下一步的行動值函數
            action_values = one_step_lookahead(s, V)
            # 選擇最佳行動
            best_a = np.argmax(action_values)
            
            # 貪婪策略：若有新的最佳行動，修改行動策略
            if chosen_a != best_a:
                policy_stable = False
            policy[s] = np.eye(n_actions)[best_a]
        
        # 如果已無較佳行動策略，則回傳策略及狀態值函數
        if policy_stable:
            return policy, V

In [156]:
# 執行策略循環
policy, v = policy_improvement(env)

In [157]:
# 顯示結果
print("策略機率分配:")
print(policy)
print("")

print("4x4 策略機率分配 (0=up, 1=right, 2=down, 3=left):")
print(np.reshape(np.argmax(policy, axis=1), (int(n_states ** 0.5), int(n_states ** 0.5))))
print("")

print("4x4 狀態值函數:")
print(v.reshape((int(n_states ** 0.5), int(n_states ** 0.5))))

策略機率分配:
[[1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]]

4x4 策略機率分配 (0=up, 1=right, 2=down, 3=left):
[[0 3 3 3]
 [0 0 0 0]
 [3 1 0 0]
 [0 2 1 0]]

4x4 狀態值函數:
[[0.8232943  0.82321549 0.82315953 0.8231305 ]
 [0.82331142 0.         0.52924366 0.        ]
 [0.8233444  0.82339085 0.76458368 0.        ]
 [0.         0.88225455 0.94112542 0.        ]]
